# Figure 6 and 7

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: GlcConst, VmaxPDH, pHleak, VmaxF1, VmaxETC, J_ANT, J_HL
using MitochondrialDynamics: G3P, Pyr, NADH_c, NADH_m, Ca_c, Ca_m, ΔΨm, ATP_c, ADP_c, degavg
using MitochondrialDynamics.Utils: second, μM, mV, mM, Hz, minute

using Plots
pyplot()
Plots.pyrcparams["font.size"] = 14
Plots.pyrcparams["font.sans-serif"] = "Arial"
Plots.pyrcparams["font.family"] = "sans-serif"

In [ ]:
glc = range(3.0mM, 30.0mM, length=301)
tend = 50minute
@named sys = make_model()
prob = SteadyStateProblem(sys, DEFAULT_U0)

## Figure 6

In [ ]:
idxGlc = findfirst(isequal(GlcConst), parameters(sys))
idxVmaxPDH = findfirst(isequal(VmaxPDH), parameters(sys))
idxpHleak = findfirst(isequal(pHleak), parameters(sys))
idxVmaxF1 = findfirst(isequal(VmaxF1), parameters(sys))
idxVmaxETC = findfirst(isequal(VmaxETC), parameters(sys))

function make_dm_prob(prob; rPDH=0.5, rETC=0.75, rHL=1.4, rF1=0.5)
    newparams = copy(prob.p)
    newparams[idxVmaxPDH] = prob.p[idxVmaxPDH] * rPDH
    newparams[idxpHleak] = prob.p[idxpHleak] * rHL
    newparams[idxVmaxF1] = prob.p[idxVmaxF1] * rF1
    newparams[idxVmaxETC] = prob.p[idxVmaxETC] * rETC
    return remake(prob, p=newparams)
end

function make_rotenone_prob(prob; rETC=0.1)
    newparams = copy(prob.p)
    newparams[idxVmaxETC] = prob.p[idxVmaxETC] * rETC
    return remake(prob, p=newparams)
end

function make_oligomycin_prob(prob; rF1=0.1)
    newparams = copy(prob.p)
    newparams[idxVmaxF1] = prob.p[idxVmaxF1] * rF1
    return remake(prob, p=newparams)
end

function make_fccp_prob(prob; rHL=5)
    newparams = copy(prob.p)
    newparams[idxpHleak] = prob.p[idxpHleak] * rHL
    return remake(prob, p=newparams)
end

In [ ]:
prob_dm = make_dm_prob(sys, prob)

sols = map(glc) do g
    p = copy(prob.p)
    p[idxGlc] = g
    solve(remake(prob; p=p), DynamicSS(Rodas5()))
end

solsDM = map(glc) do g
    p = copy(prob_dm.p)
    p[idxGlc] = g
    solve(remake(prob_dm; p=p), DynamicSS(Rodas5()))
end

glc5 = glc ./ 5

In [ ]:
g3p = [getindex.(sols, G3P) getindex.(solsDM, G3P)] .* 1000
pyr = [getindex.(sols, Pyr) getindex.(solsDM, Pyr)] .* 1000
nadh_c = [getindex.(sols, NADH_c) getindex.(solsDM, NADH_c)] .* 1000
nadh_m = [getindex.(sols, NADH_m) getindex.(solsDM, NADH_m)] .* 1000
ca_c = [getindex.(sols, Ca_c) getindex.(solsDM, Ca_c)] .* 1000
ca_m = [getindex.(sols, Ca_m) getindex.(solsDM, Ca_m)] .* 1000
dpsi = [getindex.(sols, ΔΨm) getindex.(solsDM, ΔΨm)] .* 1000
td = [getindex.(sols, ATP_c) ./ getindex.(sols, ADP_c) getindex.(solsDM, ATP_c) ./ getindex.(solsDM, ADP_c)]
avgDeg = [getindex.(sols, degavg) getindex.(solsDM, degavg)]

In [ ]:
pl1 = plot(glc5, g3p; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="Conc. (μM)", title="G3P", titlelocation=:left)
pl2 = plot(glc5, pyr; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="Conc. (μM)", title="(A) Pyruvate", titlelocation=:left)
pl3 = plot(glc5, nadh_c; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="Conc. (μM)", title="(B) Cytosolic NADH", titlelocation=:left)
pl4 = plot(glc5, nadh_m; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="Conc. (μM)", title="(C) Mitochondrial NADH", titlelocation=:left)
pl5 = plot(glc5, ca_c; labels=["Baseline" "Diabetic"],
    legend=:bottomright, ylabel="Conc. (μM)", title="(D) Cytosolic Ca", titlelocation=:left)
pl6 = plot(glc5, ca_m; labels=["Baseline" "Diabetic"],
    legend=:bottomright, ylabel="Conc. (μM)", title="(E) Mitochondrial Ca", titlelocation=:left)
pl7 = plot(glc5, dpsi; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="ΔΨ (mV)", title="(F) Mitochondrial membrane potential", titlelocation=:left)
pl8 = plot(glc5, td; labels=["Baseline" "Diabetic"],
    legend=:bottomright, ylabel="", title="(G) ATP:ADP", titlelocation=:left)
pl9 = plot(glc5, avgDeg; labels=["Baseline" "Diabetic"],
    legend=:right, ylabel="", title="(H) Average node degree", titlelocation=:left)

plot(pl1, pl2, pl3, pl4, pl5, pl6, pl7, pl8, pl9;
    size=(2000, 1000))

In [ ]:
# Uncomment to generate pdf file
# savefig("Fig6.pdf")

## Figure 7

In [ ]:
prob_fccp = make_fccp_prob(prob)
prob_rotenone = make_rotenone_prob(prob)
prob_oligomycin = make_oligomycin_prob(prob)

solsFCCP = map(glc) do g
    p = copy(prob_fccp.p)
    p[idxGlc] = g
    solve(remake(prob_fccp; p=p), DynamicSS(Rodas5()))
end

solsRot = map(glc) do g
    p = copy(prob_rotenone.p)
    p[idxGlc] = g
    solve(remake(prob_rotenone; p=p), DynamicSS(Rodas5()))
end

solsOligo = map(glc) do g
    p = copy(prob_oligomycin.p)
    p[idxGlc] = g
    solve(remake(prob_oligomycin; p=p), DynamicSS(Rodas5()))
end;


In [ ]:
# Gather ATP synthesis rate (fusion) and proton leak rate (fission)

jHL_baseline = getindex.(sols, J_HL)
jANT_baseline = getindex.(sols, J_ANT)
ff_baseline = jANT_baseline ./ jHL_baseline
jHL_dm = getindex.(solsDM, J_HL)
jANT_dm = getindex.(solsDM, J_ANT)
ff_dm =  jANT_dm ./ jHL_dm 
jHL_fccp = getindex.(solsFCCP, J_HL)
jANT_fccp = getindex.(solsFCCP, J_ANT)
ff_fccp = jANT_fccp ./ jHL_fccp 
jHL_rot = getindex.(solsRot, J_HL)
jANT_rot = getindex.(solsRot, J_ANT)
ff_rot = jANT_rot ./ jHL_rot  
jHL_oligo = getindex.(solsOligo, J_HL)
jANT_oligo = getindex.(solsOligo, J_ANT)
ff_oligo = jANT_oligo ./ jHL_oligo  

In [ ]:
glc5 = glc ./ 5
pl1 = plot(glc5, ff_baseline, line=(:blue, :dash), label="Baseline")
plot!(pl1, glc5, ff_dm, line=(:red, :dash), label="Diabetic")
plot!(pl1, glc5, ff_rot, line=(:green, :dash), label="Rotenone")
plot!(pl1, glc5, ff_oligo, line=(:cyan, :dash), label="Oligomycin")
plot!(pl1, glc5, ff_fccp, line=(:black, :dash), label="Uncoupler")
plot!(pl1, xlabel="Glucose (X)", ylabel="Fusion rate / Fission rate", xlim=(0.0, 6.0), ylim=(0.0, 2.5), title="(A)", titlelocation=:left)

In [ ]:
pl2 = plot(jHL_baseline, jANT_baseline, line=(:blue, :dash), label="Baseline")
scatter!(pl2, jHL_baseline[begin:10:end], jANT_baseline[begin:10:end], marker = (:blue, 1), lab=false)
plot!(pl2, jHL_dm, jANT_dm, line=(:red, :dash), label="Diabetic")
scatter!(pl2, jHL_dm[begin:10:end], jANT_dm[begin:10:end], marker = (:red, 1), lab=false)
plot!(pl2, jHL_rot, jANT_rot, line=(:green, :dash), label="Rotenone")
scatter!(pl2, jHL_rot[begin:10:end], jANT_rot[begin:10:end], marker = (:green, 1), lab=false)
plot!(pl2, jHL_oligo, jANT_oligo, line=(:cyan, :dash), label="Oligomycin")
scatter!(pl2, jHL_oligo[begin:10:end], jANT_oligo[begin:10:end], marker = (:cyan, 1), lab=false)
plot!(pl2, jHL_fccp, jANT_fccp, line=(:black, :dash), label="Uncoupler")
scatter!(pl2, jHL_fccp[begin:10:end], jANT_fccp[begin:10:end], marker = (:black, 1), lab=false)
plot!(pl2, xlabel="Proton leak rate (mM/s)", ylabel="ATP synthase rate (mM/s)", xlim=(0.0, 0.45), ylim=(0.0,0.15), title="(B)", titlelocation=:left)

In [ ]:
plot(pl1, pl2, size=(1200, 600))

In [ ]:
# Uncomment to generate pdf figure
# savefig("Fig7.pdf")